In [118]:
import scipy.io
import numpy as np
from sklearn import linear_model
from scipy import linalg as lng
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [119]:
def centerData(x):
# function to center the data

    x_c = x-np.mean(x)
    
    return x_c

In [120]:
def standardizeData(x):
    # function to normalize the data
    
    std_dev = np.std(x)
    std_dev_c = np.where(std_dev==0, 1, std_dev) 
    x_n = (np.transpose(x)-np.mean(x, axis=1)).transpose()/std_dev_c
    
    return x_n

In [182]:
def center(X):
    """ Center the columns (variables) of a data matrix to zero mean.
        
        X, MU = center(X) centers the observations of a data matrix such that each variable
        (column) has zero mean and also returns a vector MU of mean values for each variable.
     """ 
    n = X.shape[0]
    mu = np.mean(X,0)
    X = X - mu
    return X, mu

In [183]:
def normalize(X):
    """Normalize the columns (variables) of a data matrix to unit Euclidean length.
    X, MU, D = normalize(X)
    i) centers and scales the observations of a data matrix such
    that each variable (column) has unit Euclidean length. For a normalized matrix X,
    X'*X is equivalent to the correlation matrix of X.
    ii) returns a vector MU of mean values for each variable.
    iii) returns a vector D containing the Euclidean lengths for each original variable.
    
    See also CENTER
    """
    
    n = np.size(X, 0)
    X, mu = center(X)
    d = np.linalg.norm(X, ord = 2, axis = 0)
    d[np.where(d==0)] = 1
    X = np.divide(X, np.ones((n,1)) * d)
    return X, mu, d

In [184]:
def normalizetest(X,mx,d):
    """Normalize the observations of a test data matrix given the mean mx and variance varx of the training.
       X = normalizetest(X,mx,varx) centers and scales the observations of a data matrix such that each variable
       (column) has unit length.
       Returns X: the normalized test data"""
    
    n = X.shape[0]
    X = np.divide(np.subtract(X, np.ones((n,1))*mx), np.ones((n,1)) * d)
    return X

In [185]:
def ridgeMulti(X, _lambda, p, y):
    inner_prod = np.linalg.inv(np.matmul(X.T, X) + _lambda * np.eye(p,p))
    outer_prod = np.matmul(X.T, y)
    betas = np.matmul(inner_prod, outer_prod)
    return betas

In [186]:
path = r'M:\Documents\Courses\Credits\2019August\Module3\Exercises 3\Python'
mat = scipy.io.loadmat(path + '\\sand.mat')
X = mat['X']
y = mat['Y'].ravel()

# standardize X, center y
X_standard = standardizeData(X)
y_center = centerData(y)


In [187]:
y

array([2.68, 2.55, 2.55, 4.83, 4.62, 4.51, 7.3 , 7.02, 6.85, 2.34, 2.06,
       2.12, 4.74, 4.82, 4.67, 6.77, 6.56, 6.43, 1.41, 1.26, 1.37, 2.04,
       1.94, 2.16, 2.63, 2.63, 2.22, 2.69, 2.55, 2.69, 3.76, 3.71, 3.6 ,
       4.58, 4.35, 3.93, 3.92, 3.6 , 4.85, 4.84, 4.71, 5.52, 5.71, 6.03,
       6.6 , 6.41, 6.23, 3.88, 4.57, 4.11, 7.47, 7.57, 7.64, 9.15, 9.17,
       8.82, 4.69, 4.67, 4.67])

# Question1

In [188]:
# 1(a) - Cp statistic

Y_OLS = np.matmul(X, (linalg.lstsq(X,y))[0]) # NOTE: OLS solution doesn't make sense for p>>n
s2 = ((Y_OLS - y)**2).sum(axis=0) # add n for variance estimate
# NOTE: Check the value of s2 - our estimate of the variance of the noise in data is numerically zero
# so what we are saying with the Cp is: we believe data has no noise. Go ahead and make as complicated a model
# as possible. Cp works when n > p.


[n, p] = np.shape(X)

off = np.ones(n)
M = np.c_[off, X] # Include offset / intercept


# Linear solver
beta, _, rnk, s = lng.lstsq(X, y)

yhat = np.matmul(X, beta)

e = y - np.matmul(X, beta) # Low bias std
s = np.std(e)

k = len(y)

Cp = np.zeros(k)

Beta = np.zeros((p,n))

for j in range(k):
    regr = linear_model.Lars(n_nonzero_coefs=j)
    regr.fit(X, y)
    Beta[:,j] = regr.coef_
    yHat = np.matmul(X, Beta[:,j])
    
    residuals = (y-yHat) 
    s = residuals.std()
    
    Cp[j] = (1/(s**2))*sum((y-yHat)**2) - n + 2*j
    
#for i in range(np.shape(Beta)[0]):
#    plt.plot(Beta[i,:])
    
#plt.legend()
#plt.plot(Cp)    



In [190]:
# 1(b)

# split data into train and test

CV = 5 # if CV = n leave-one-out, you may try different numbers
# this corresponds to crossvalind in matlab
# permutes observations - useful when n != 0
I = np.asarray([0] * n)
for i in range(n):
    I[i] = (i + 1) % CV + 1
     
I = I[np.random.permutation(n)]

mse_train = np.zeros((K, p))
mse_test = np.zeros((K, p))

for i in range(1, CV+1):
    X_train = X[i!=I,:]
    X_test = X[i==I,:]
    y_train = y[i!=I]
    y_test = y[i==I]
    
    # normalize X
    X_train_norm, mx, varx = normalize(X_train) # normalize training data
    X_test_norm = normalizetest(X_test, mx, varx) # use mean and variance of training data for normalizing test data
    
    
    y_train_center, my = center(y_train) # center training response
    
    #Ytr = Ytr[0,:] # Indexing in python thingy, no time to solve it
    y_test_center = y_test-my # use the mean value of the training response to center the test response
    
    # create a model on the training data
    K = p
    Betas = np.zeros((K, p))
    for j in range(K):
        
        reg = linear_model.Lars(n_nonzero_coefs=j, fit_path = False, fit_intercept = False, verbose = True)
        reg.fit(X_train_norm,y_train_center)
        beta = reg.coef_.ravel()
        Betas[i-1, :] = beta
        
        #reg = linear_model.Lars(n_nonzero_coefs=j) # LARS model
        #reg.fit(X_train_norm, y_train_center)
        
        #beta = reg.coef_
        
        y_train_hat = np.matmul(X_train_norm, beta)
        y_test_hat = np.matmul(X_test_norm, beta)
        
        mse_train[i-1,j] = np.mean(np.sqrt((y_train_center-y_train_hat)**2))
        mse_test[i-1,j] = np.mean(np.sqrt((y_test_center-y_test_hat)**2))
        

KeyboardInterrupt: 

# Question2

In [117]:
# 2(a)
reg.coef_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [116]:
y_train_center.shape

(48, 48)